# DDAD - Dense Depth for Autonomous Driving

DDAD is a new autonomous driving benchmark from TRI (Toyota Research Institute) for long range (up to 250m) and dense depth estimation in challenging and diverse urban conditions. It contains monocular videos and accurate ground-truth depth (across a full 360 degree field of view) generated from high-density LiDARs mounted on a fleet of self-driving cars operating in a cross-continental setting. DDAD contains scenes from urban settings in the United States (San Francisco, Bay Area, Cambridge, Detroit, Ann Arbor) and Japan (Tokyo, Odaiba). This notebook will demonstrate a number of simple steps that will allow you to load and visualize the DDAD dataset.


In [ ]:
import cv2
import numpy as np
import PIL
from IPython import display
from matplotlib.cm import get_cmap

from dgp.datasets.synchronized_dataset import SynchronizedSceneDataset
from dgp.proto.ontology_pb2 import Ontology
from dgp.utils.protobuf import open_pbobject
from dgp.utils.visualization import visualize_semantic_segmentation_2d

plasma_color_map = get_cmap('plasma')

In [ ]:
# Define high level variables
DDAD_TRAIN_VAL_JSON_PATH = '/data/datasets/ddad_train_val/ddad.json'
DDAD_TEST_JSON_PATH = '/data/datasets/ddad_test/ddad_test.json'
DATUMS = ['lidar'] + ['CAMERA_%02d' % idx for idx in [1, 5, 6, 7, 8, 9]] 

## DDAD Train split

The training set contains 150 scenes with a total of 12650 individual samples (75900 RGB images).

In [ ]:
ddad_train = SynchronizedSceneDataset(
    DDAD_TRAIN_VAL_JSON_PATH,
    split='train',
    datum_names=DATUMS,
    generate_depth_from_datum='lidar'
)
print('Loaded DDAD train split containing {} samples'.format(len(ddad_train)))

### Load a random sample

In [ ]:
random_sample_idx = np.random.randint(len(ddad_train))
sample = ddad_train[random_sample_idx] # scene[0] - lidar, scene[1:] - camera datums
sample_datum_names = [datum['datum_name'] for datum in sample]
print('Loaded sample {} with datums {}'.format(random_sample_idx, sample_datum_names))

### Visualize camera images

In [ ]:
# Concat images and visualize
images = [cam['rgb'].resize((192,120), PIL.Image.BILINEAR) for cam in sample[1:]]
images = np.concatenate(images, axis=1)
display.display(PIL.Image.fromarray(images))

### Visualize corresponding depths

In [ ]:
# Visualize corresponding depths, if the depth has been projected into the camera images
if 'depth' in sample[1].keys():
    # Load and resize depth images
    depths = [cv2.resize(cam['depth'], dsize=(192,120), interpolation=cv2.INTER_NEAREST) \
              for cam in sample[1:]]
    # Convert to RGB for visualization
    depths = [plasma_color_map(d)[:,:,:3] for d in depths]
    depths = np.concatenate(depths, axis=1)
    display.display(PIL.Image.fromarray((depths*255).astype(np.uint8)))

### Visualize Lidar

In [ ]:
# Note: this requires open3d
import open3d as o3d

# Get lidar point cloud from sample
lidar_cloud = sample[0]['point_cloud']
# Create open3d visualization objects
o3d_colors = np.tile(np.array([0., 0., 0.]), (len(lidar_cloud), 1))
o3d_cloud = o3d.geometry.PointCloud()
o3d_cloud.points = o3d.utility.Vector3dVector(lidar_cloud)
o3d_cloud.colors = o3d.utility.Vector3dVector(o3d_colors)
# Visualize (Note: needs open3d, openGL and X server support)
o3d.visualization.draw_geometries([o3d_cloud])

## DDAD train with temporal context

To also return temporally adjacent scenes, use `forward_context` and `backward_context`.

In [ ]:
# Intantiate dataset with forward and backward context

ddad_train_with_context = SynchronizedSceneDataset(
    DDAD_TRAIN_VAL_JSON_PATH,
    split='train',
    datum_names=('CAMERA_01',),
    generate_depth_from_datum='lidar',
    forward_context=1, 
    backward_context=1
)

### Visualize front camera images

In [ ]:
# Load random sample
# Note that when forward_context or backward_context is used, the loader returns a list of samples
samples = ddad_train_with_context[np.random.randint(len(ddad_train))] 
front_cam_images = []
for sample in samples:
    front_cam_images.append(sample[0]['rgb'])
# Resize images and visualize
front_cam_images = [img.resize((192,120), PIL.Image.BILINEAR) for img in front_cam_images]
front_cam_images = np.concatenate(front_cam_images, axis=1)
display.display(PIL.Image.fromarray(front_cam_images))

## DDAD Val split

The validation set contains 50 scenes with a total of 3950 individual samples.

In [ ]:
# Load the val set
ddad_val = SynchronizedSceneDataset(
    DDAD_TRAIN_VAL_JSON_PATH,
    split='val',
    datum_names=DATUMS,
    generate_depth_from_datum='lidar'
)
print('Loaded DDAD val split containing {} samples'.format(len(ddad_val)))

### Load the panoptic segmentation labels from the val set

50 of the DDAD validation samples have panoptic segmentation annotations for the front camera images. These annotations can be used for detailed, per-class evaluation.

In [ ]:
ddad_val = SynchronizedSceneDataset(
    DDAD_TRAIN_VAL_JSON_PATH,
    split='val',
    datum_names=('CAMERA_01',),
    requested_annotations=('semantic_segmentation_2d', 'instance_segmentation_2d'),
    only_annotated_datums=True
)
print('Loaded annotated samples from DDAD val split. Total samples: {}.'.format(len(ddad_val)))

# Visualize the semantic segmentation labels

In [ ]:
# Load instance and semantic segmentation ontologies
semseg_ontology = open_pbobject(ddad_val.scenes[0].ontology_files['semantic_segmentation_2d'], Ontology)
instance_ontology = open_pbobject(ddad_val.scenes[0].ontology_files['instance_segmentation_2d'], Ontology)

# Load random sample 
random_sample_idx = np.random.randint(len(ddad_val))
sample = ddad_val[random_sample_idx]

# Get image and sample  segmentation annotation from sample 
image = np.array(sample[0]['rgb'])
semantic_segmentation_2d_annotation = sample [0]['semantic_segmentation_2d']
sem_seg_image = visualize_semantic_segmentation_2d(
        semantic_segmentation_2d_annotation, semseg_ontology, image=image, debug=False
)

# Visualize
image = cv2.resize(image, dsize=(320,240), interpolation=cv2.INTER_NEAREST)
sem_seg_image = cv2.resize(sem_seg_image, dsize=(320,240), interpolation=cv2.INTER_NEAREST)
display.display(PIL.Image.fromarray(np.concatenate([image, sem_seg_image], axis=1)))

## DDAD Test split

In [ ]:
ddad_test = SynchronizedSceneDataset(
    DDAD_TEST_JSON_PATH,
    split='test',
    datum_names=DATUMS,
    generate_depth_from_datum='lidar'
)
print('Loaded DDAD test split containing {} samples'.format(len(ddad_test)))